Comenzamos realizando la misma práctica que hicimos en Hive en Spark, importando el csv. Sería recomendable intentarlo con opciones que quiten las "" de los campos, que ignoren los espacios innecesarios en los campos, que sustituyan los valores vacíos por 0 y que infiera el esquema.

In [0]:
%scala
import sqlContext.implicits._
import org.apache.spark.sql.functions._

val padronDf = spark.read
    .format("csv")
    .option("header", true)
    .option("inferSchema", true)
    .option("samplingRatio", 0.1)
    .option("delimiter", ";")
    .load("dbfs:/FileStore/shared_uploads/giovanni.rodriguez@bosonit.com/Rango_Edades.csv")
    .withColumn("DESC_DISTRITO", trim($"DESC_DISTRITO"))
    .withColumn("DESC_BARRIO", trim($"DESC_BARRIO"))
    .withColumn("EspanolesHombres", when(col("EspanolesHombres").isNull, 0).otherwise($"EspanolesHombres"))
    .withColumn("EspanolesHombres", when(col("EspanolesHombres").isNull, 0).otherwise($"EspanolesHombres"))
    .withColumn("EspanolesMujeres", when(col("EspanolesMujeres").isNull, 0).otherwise($"EspanolesMujeres"))
    .withColumn("ExtranjerosHombres", when(col("ExtranjerosHombres").isNull, 0).otherwise($"ExtranjerosHombres"))
    .withColumn("ExtranjerosMujeres", when(col("ExtranjerosMujeres").isNull, 0).otherwise($"ExtranjerosMujeres"))

import sqlContext.implicits._
import org.apache.spark.sql.functions._
padronDf: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]

Enumera todos los barrios diferentes.

In [0]:
%scala
padronDf.select("DESC_BARRIO").distinct().show()

+----------------+
 DESC_BARRIO|
+----------------+
 VALVERDE|
 CORTES|
 TRAFALGAR|
FUENTE DEL BERRO|
 PACIFICO|
 CUATRO CAMINOS|
 EL PILAR|
 ARGUELLES|
 VALDEACEDERAS|
 VALLEHERMOSO|
 IBIZA|
 EMBAJADORES|
 CASTELLANA|
 HISPANOAMERICA|
 ACACIAS|
 LOS CARMENES|
 CHOPERA|
 PALACIO|
 JUSTICIA|
 UNIVERSIDAD|
+----------------+
only showing top 20 rows

Crea una vista temporal de nombre "padron" y a través de ella cuenta el número de barrios diferentes que hay.

In [0]:
%scala
padronDf.createOrReplaceTempView("padron")
spark.sql("SELECT DISTINCT count(COD_BARRIO) as Barrios FROM padron").show()

+-------+
Barrios|
+-------+
 238304|
+-------+

Crea una nueva columna que muestre la longitud de los campos de la columna DESC_DISTRITO y que se llame "longitud".

In [0]:
%scala
val padronLengthDf = padronDf.withColumn("longitud", length(col("DESC_DISTRITO")))
padronLengthDf.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 0| 3| 1| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 1| 5| 2| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 2| 2| 2| 2| 3| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 3| 3| 1| 0| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 4| 3| 1| 1| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 5| 1| 1| 0| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 6| 1| 2| 1| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 7| 1| 1| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 8| 3| 2| 1| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 9| 2| 0| 0| 2| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 10| 1| 2| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 11| 5| 1| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 12| 2| 4| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 13| 5| 4| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 14| 1| 0| 0| 2| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 15| 1| 3| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 16| 6| 4| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 17| 2| 2| 0| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 18| 3| 5| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 19| 3| 4| 2| 2| 6|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
only showing top 20 rows

padronLengthDf: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]

Crea una nueva columna que muestre el valor 5 para cada uno de los registros de la tabla.

In [0]:
%scala
val padronValueDf = padronLengthDf.withColumn("valor", lit(5))
padronValueDf.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+-----+
COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|valor|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+-----+
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 0| 3| 1| 1| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 1| 5| 2| 0| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 2| 2| 2| 2| 3| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 3| 3| 1| 0| 1| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 4| 3| 1| 1| 1| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 5| 1| 1| 0| 1| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 6| 1| 2| 1| 1| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 7| 1| 1| 1| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 8| 3| 2| 1| 1| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 9| 2| 0| 0| 2| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 10| 1| 2| 1| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 11| 5| 1| 1| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 12| 2| 4| 0| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 13| 5| 4| 0| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 14| 1| 0| 0| 2| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 15| 1| 3| 1| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 16| 6| 4| 0| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 17| 2| 2| 0| 1| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 18| 3| 5| 1| 0| 6| 5|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 19| 3| 4| 2| 2| 6| 5|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+-----+
only showing top 20 rows

padronValueDf: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 12 more fields]

Borra esta columna.

In [0]:
%scala
val padronValueDropDf = padronValueDf.drop("valor")
padronValueDropDf.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 0| 3| 1| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 1| 5| 2| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 2| 2| 2| 2| 3| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 3| 3| 1| 0| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 4| 3| 1| 1| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 5| 1| 1| 0| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 6| 1| 2| 1| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 7| 1| 1| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 8| 3| 2| 1| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 9| 2| 0| 0| 2| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 10| 1| 2| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 11| 5| 1| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 12| 2| 4| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 13| 5| 4| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 14| 1| 0| 0| 2| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 15| 1| 3| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 16| 6| 4| 0| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 17| 2| 2| 0| 1| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 18| 3| 5| 1| 0| 6|
 1| CENTRO| 101| PALACIO| 1| 1001| 1| 19| 3| 4| 2| 2| 6|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
only showing top 20 rows

padronValueDropDf: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]

Particiona el DataFrame por las variables DESC_DISTRITO y DESC_BARRIO. Almacénalo en caché.

In [0]:
%scala
val padronParticionado = padronValueDropDf.repartition($"DESC_DISTRITO", $"DESC_BARRIO")
padronParticionado.cache()

padronParticionado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]
res87: padronParticionado.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]

Lanza una consulta contra el DF resultante en la que muestre el número total de "espanoleshombres", "espanolesmujeres", extranjeroshombres" y "extranjerosmujeres" para cada barrio de cada distrito. Las columnas distrito y barrio deben ser las primeras en aparecer en el show. Los resultados deben estar ordenados en orden de más a menos según la columna "extranjerosmujeres" y desempatarán por la columna "extranjeroshombres".

In [0]:
%scala
padronParticionado.groupBy("DESC_DISTRITO", "DESC_BARRIO").agg(count("EspanolesHombres") as "EspanolesHombres", count("EspanolesMujeres")  as "EspanolesMujeres", count("ExtranjerosHombres") as "ExtranjerosHombres", count("ExtranjerosMujeres") as "ExtranjerosMujeres").orderBy(desc("EspanolesMujeres")).show()

+-------------------+--------------------+----------------+----------------+------------------+------------------+
 DESC_DISTRITO| DESC_BARRIO|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+-------------------+--------------------+----------------+----------------+------------------+------------------+
 LATINA| ALUCHE| 5750| 5750| 5750| 5750|
 CIUDAD LINEAL| PUEBLO NUEVO| 4498| 4498| 4498| 4498|
FUENCARRAL-EL PARDO| EL PILAR| 4370| 4370| 4370| 4370|
 LATINA| LAS AGUILAS| 4100| 4100| 4100| 4100|
 HORTALEZA| PINAR DEL REY| 4078| 4078| 4078| 4078|
 CIUDAD LINEAL| VENTAS| 4026| 4026| 4026| 4026|
FUENCARRAL-EL PARDO| VALVERDE| 3801| 3801| 3801| 3801|
 LATINA| PUERTA DEL ANGEL| 3630| 3630| 3630| 3630|
FUENCARRAL-EL PARDO| PE�A GRANDE| 3513| 3513| 3513| 3513|
 CENTRO| EMBAJADORES| 3485| 3485| 3485| 3485|
 SALAMANCA| GUINDALERA| 3364| 3364| 3364| 3364|
 PUENTE DE VALLECAS| PALOMERAS SURESTE| 3363| 3363| 3363| 3363|
 CARABANCHEL| VISTA ALEGRE| 3331| 3331| 3331| 3331|
 PUENTE DE VALLECAS| NUMANCIA| 3320| 3320| 3320| 3320|
 VILLAVERDE|VILLAVERDE ALTO C.H.| 3196| 3196| 3196| 3196|
 HORTALEZA| CANILLAS| 3021| 3021| 3021| 3021|
 PUENTE DE VALLECAS| SAN DIEGO| 2906| 2906| 2906| 2906|
 PUENTE DE VALLECAS| ENTREVIAS| 2904| 2904| 2904| 2904|
 TETUAN| CUATRO CAMINOS| 2873| 2873| 2873| 2873|
 LATINA| LUCERO| 2851| 2851| 2851| 2851|
+-------------------+--------------------+----------------+----------------+------------------+------------------+
only showing top 20 rows

Elimina el registro en caché.

In [0]:
%scala
padronParticionado.unpersist()

res90: padronParticionado.type = [COD_DISTRITO: int, DESC_DISTRITO: string ... 11 more fields]

Crea un nuevo DataFrame a partir del original que muestre únicamente una columna con DESC_BARRIO, otra con DESC_DISTRITO y otra con el número total de "espanoleshombres" residentes en cada distrito de cada barrio. Únelo (con un join) con el DataFrame original a través de las columnas en común.

In [0]:
%scala
val countDf = spark.sql("SELECT DESC_BARRIO, DESC_DISTRITO, count(espanolesHombres) as EspanolesHombres FROM padron GROUP BY DESC_BARRIO, DESC_DISTRITO")

padronDf.join(countDf, "DESC_DISTRITO")

countDf: org.apache.spark.sql.DataFrame = [DESC_BARRIO: string, DESC_DISTRITO: string ... 1 more field]
res91: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, COD_DISTRITO: int ... 12 more fields]

Repite la función anterior utilizando funciones de ventana. (over(Window.partitionBy.....)).

In [0]:
%scala
import org.apache.spark.sql.expressions.Window

val window = Window.partitionBy("DESC_BARRIO", "DESC_DISTRITO")
padronParticionado.withColumn("EspanolesHombres", count("espanolesHombres").over(window)).show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|longitud|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 0| 2031| 4| 1| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 1| 2031| 3| 1| 1| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 2| 2031| 3| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 3| 2031| 1| 3| 2| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 4| 2031| 1| 1| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 5| 2031| 6| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 6| 2031| 3| 1| 1| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 7| 2031| 2| 1| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 8| 2031| 4| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 9| 2031| 9| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 10| 2031| 5| 1| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 11| 2031| 6| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 12| 2031| 7| 1| 1| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 13| 2031| 5| 1| 1| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 14| 2031| 3| 0| 1| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 15| 2031| 8| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 16| 2031| 3| 0| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 17| 2031| 7| 1| 0| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 18| 2031| 6| 1| 1| 11|
 11| CARABANCHEL| 1107| ABRANTES| 7| 11157| 157| 19| 2031| 3| 0| 0| 11|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+--------+
only showing top 20 rows

import org.apache.spark.sql.expressions.Window
window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@4041a4a8

Mediante una función Pivot muestra una tabla (que va a ser una tabla de contingencia) que contenga los valores totales (la suma de valores) de espanolesmujeres para cada distrito y en cada rango de edad (COD_EDAD_INT). Los distritos incluidos deben ser únicamente CENTRO, BARAJAS y RETIRO y deben figurar como columnas.

In [0]:
%scala
val padronEdadesDistrito = padronParticionado.where($"DESC_DISTRITO".isin("CENTRO", "BARAJAS", "RETIRO"))
                           .groupBy("COD_EDAD_INT")
                           .pivot("DESC_DISTRITO")
                           .agg(sum($"espanolesMujeres"))
                           .orderBy("COD_EDAD_INT")
padronEdadesDistrito.show()

+------------+-------+------+------+
COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|
+------------+-------+------+------+
 0| 146| 262| 317|
 1| 160| 240| 318|
 2| 171| 200| 377|
 3| 175| 216| 365|
 4| 201| 239| 412|
 5| 217| 224| 420|
 6| 242| 231| 405|
 7| 229| 237| 466|
 8| 234| 228| 417|
 9| 236| 235| 421|
 10| 260| 252| 419|
 11| 241| 231| 423|
 12| 259| 244| 417|
 13| 267| 274| 443|
 14| 250| 249| 407|
 15| 269| 251| 429|
 16| 269| 267| 422|
 17| 265| 261| 417|
 18| 223| 285| 455|
 19| 254| 304| 411|
+------------+-------+------+------+
only showing top 20 rows

padronEdadesDistrito: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_EDAD_INT: int, BARAJAS: bigint ... 2 more fields]

Utilizando este nuevo DF, crea 3 columnas nuevas que hagan referencia a qué porcentaje de la suma de "espanolesmujeres" en los tres distritos para cada rango de edad representa cada uno de los tres distritos. Debe estar redondeada a 2 decimales.

In [0]:
%scala
padronEdadesDistrito.withColumn("totalEdades", expr("CENTRO + BARAJAS + RETIRO"))
                    .withColumn("AvgBarajas", expr("CAST(BARAJAS * 100 / totalEdades AS DECIMAL(38, 2))"))
                    .withColumn("AvgCentro", expr("CAST(CENTRO * 100 / totalEdades AS DECIMAL(38, 2))"))
                    .withColumn("AvgRetiro", expr("CAST(RETIRO * 100 / totalEdades AS DECIMAL(38, 2))"))
.drop("totalEdades").show()

+------------+-------+------+------+----------+---------+---------+
COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|AvgBarajas|AvgCentro|AvgRetiro|
+------------+-------+------+------+----------+---------+---------+
 0| 146| 262| 317| 20.14| 36.14| 43.72|
 1| 160| 240| 318| 22.28| 33.43| 44.29|
 2| 171| 200| 377| 22.86| 26.74| 50.40|
 3| 175| 216| 365| 23.15| 28.57| 48.28|
 4| 201| 239| 412| 23.59| 28.05| 48.36|
 5| 217| 224| 420| 25.20| 26.02| 48.78|
 6| 242| 231| 405| 27.56| 26.31| 46.13|
 7| 229| 237| 466| 24.57| 25.43| 50.00|
 8| 234| 228| 417| 26.62| 25.94| 47.44|
 9| 236| 235| 421| 26.46| 26.35| 47.20|
 10| 260| 252| 419| 27.93| 27.07| 45.01|
 11| 241| 231| 423| 26.93| 25.81| 47.26|
 12| 259| 244| 417| 28.15| 26.52| 45.33|
 13| 267| 274| 443| 27.13| 27.85| 45.02|
 14| 250| 249| 407| 27.59| 27.48| 44.92|
 15| 269| 251| 429| 28.35| 26.45| 45.21|
 16| 269| 267| 422| 28.08| 27.87| 44.05|
 17| 265| 261| 417| 28.10| 27.68| 44.22|
 18| 223| 285| 455| 23.16| 29.60| 47.25|
 19| 254| 304| 411| 26.21| 31.37| 42.41|
+------------+-------+------+------+----------+---------+---------+
only showing top 20 rows

Guarda el archivo csv original particionado por distrito y por barrio (en ese orden) en un directorio local. Consulta el directorio para ver la estructura de los ficheros y comprueba que es la esperada.

In [0]:
%scala
padronDf.write.format("csv").partitionBy("DESC_DISTRITO", "DESC_BARRIO").save("/tmp/padron")

In [0]:
%fs ls /tmp/padron

path,name,size,modificationTime
dbfs:/tmp/padron/DESC_DISTRITO=ARGANZUELA/,DESC_DISTRITO=ARGANZUELA/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=BARAJAS/,DESC_DISTRITO=BARAJAS/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=CARABANCHEL/,DESC_DISTRITO=CARABANCHEL/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=CENTRO/,DESC_DISTRITO=CENTRO/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=CHAMARTIN/,DESC_DISTRITO=CHAMARTIN/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=CHAMBERI/,DESC_DISTRITO=CHAMBERI/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=CIUDAD LINEAL/,DESC_DISTRITO=CIUDAD LINEAL/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=FUENCARRAL-EL PARDO/,DESC_DISTRITO=FUENCARRAL-EL PARDO/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=HORTALEZA/,DESC_DISTRITO=HORTALEZA/,0,0
dbfs:/tmp/padron/DESC_DISTRITO=LATINA/,DESC_DISTRITO=LATINA/,0,0
